# <center> Lista 5

### Objetivo: realizar a predição para obter o preço de venda de um imóvel a partir do seguinte dataset "Boston House Price Dataset". 

In [85]:
import numpy as np
import pandas as pd
import matplotlib as plt
from ml_libs import Regressao_linear_gd
from ml_libs import metricas
from ml_libs import Normalizacao
from importlib import reload

In [86]:
reload(Regressao_linear_gd)
reload(metricas)

<module 'ml_libs.metricas' from '/home/michel/Documentos/Machine-Learning/Lista 5/ml_libs/metricas.py'>

In [3]:
cols = ['CRIM ','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
df = pd.read_csv('housing.csv',names=cols, sep='\s+')

In [4]:
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


### Informações sobre Atributos:

    01. CRIM    Taxa de criminalidade per capita por cidade
    02. ZN      proporção de terrenos residenciais zoneada para lotes mais 25.000 pés quadrados
    03. INDUS   proporção de hectares comerciais não varejistas por cidade
    04. CHAS    Variável dummy Charles River (= 1 se limites do setor rio; 0 caso contrário)
    05. NOX     Concentração de óxidos nítricos (partes por 10 milhões)
    06. RM      média de quartos por habitação
    07. AGE     Proporção de unidades ocupadas pelo proprietário construídas antes de 1940
    08. DIS     Distâncias ponderadas pelo para cinco centros de emprego em Boston
    09. RAD     Índice de acessibilidade a rodovias radiais
    10. TAX     TAXA DE IMPOSTO DE IMPOSIÇÃO DE VALOR TOTAL POR 10.000
    11. PTRATIO Proporção aluno-professor por cidade
    12. B       1000(Bk - 0,63)^2 onde Bk é a proporção de negros pela cidade
    13. LSTAT   % Status menor da população
    14. MEDV    Valor mediano de residências ocupadas pelo proprietário em US$1000

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null int64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null int64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(12), int64(2)
memory usage: 55.4 KB


In [6]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [7]:
df.hist(bins=20, figsize=(15,15));

In [8]:
df.corr().style.set_precision(4)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
CRIM,1,-0.2005,0.4066,-0.05589,0.421,-0.2192,0.3527,-0.3797,0.6255,0.5828,0.2899,-0.3851,0.4556,-0.3883
ZN,-0.2005,1,-0.5338,-0.0427,-0.5166,0.312,-0.5695,0.6644,-0.3119,-0.3146,-0.3917,0.1755,-0.413,0.3604
INDUS,0.4066,-0.5338,1,0.06294,0.7637,-0.3917,0.6448,-0.708,0.5951,0.7208,0.3832,-0.357,0.6038,-0.4837
CHAS,-0.05589,-0.0427,0.06294,1,0.0912,0.09125,0.08652,-0.09918,-0.007368,-0.03559,-0.1215,0.04879,-0.05393,0.1753
NOX,0.421,-0.5166,0.7637,0.0912,1,-0.3022,0.7315,-0.7692,0.6114,0.668,0.1889,-0.3801,0.5909,-0.4273
RM,-0.2192,0.312,-0.3917,0.09125,-0.3022,1,-0.2403,0.2052,-0.2098,-0.292,-0.3555,0.1281,-0.6138,0.6954
AGE,0.3527,-0.5695,0.6448,0.08652,0.7315,-0.2403,1,-0.7479,0.456,0.5065,0.2615,-0.2735,0.6023,-0.377
DIS,-0.3797,0.6644,-0.708,-0.09918,-0.7692,0.2052,-0.7479,1,-0.4946,-0.5344,-0.2325,0.2915,-0.497,0.2499
RAD,0.6255,-0.3119,0.5951,-0.007368,0.6114,-0.2098,0.456,-0.4946,1,0.9102,0.4647,-0.4444,0.4887,-0.3816
TAX,0.5828,-0.3146,0.7208,-0.03559,0.668,-0.292,0.5065,-0.5344,0.9102,1,0.4609,-0.4418,0.544,-0.4685


### Selecionando as features X e label y

In [9]:
X = df.values[:, :-1]
y = df.values[:, -1]

### Randomização dos dados usando semente (seed)

In [34]:
rs = np.random.RandomState(1)
shuffled_indices = rs.permutation(X.shape[0])

In [11]:
X = X[shuffled_indices]
y = y[shuffled_indices]

In [13]:
X[:1]

array([[4.9320e-02, 3.3000e+01, 2.1800e+00, 0.0000e+00, 4.7200e-01,
        6.8490e+00, 7.0300e+01, 3.1827e+00, 7.0000e+00, 2.2200e+02,
        1.8400e+01, 3.9690e+02, 7.5300e+00]])

In [81]:
y[:5]

array([28.2, 23.9, 16.6, 22. , 20.8])

### Selecionando 70% dos dados para treino e 30% para teste

In [49]:
n = int(np.count_nonzero(y) * 0.70)
X_train = X[:n]
y_train = y[:n]
X_test = X[n:]
y_test = y[n:]

#### Verificacao de dados

In [50]:
np.average(X_train), np.average(X_test), np.average(y_train), np.average(y_test)

(69.99891393741851, 70.24876178137652, 22.5274011299435, 22.545394736842105)

(69.73880963494133, 70.00534072874494, 22.5274011299435, 22.545394736842105)

# Criação de modelos e predições

## Sem normalização e Standardização

#### Regressão Linear com Gradiente Descendente

In [35]:
model = Regressao_linear_gd.LinearRegressionGD(eta = 0.1, n_iter = 20)

In [36]:
model.fit(X_train,y_train);

In [37]:
y_predict = model.predict(X_test);

In [51]:
rlgd_metric_mse = metricas.mse(y_test,y_predict)
rlgd_metric_rmse = metricas.rmse(y_test,y_predict)

In [53]:
rlgd_metric_rmse

1.6553688780460185e+142

### Regressão Linear do Scikit-Learning

In [70]:
from sklearn.linear_model import LinearRegression

In [55]:
model_sk = LinearRegression()

In [60]:
model_sk.fit(X_train,y_train);

In [76]:
skmetric = model_sk.predict(X_test)
metricas.rmse(skmetric,y_test)

5.426876664838224

### Gradiente Descendente Estocástico Regressor do Scikit-Learning

In [62]:
from sklearn.linear_model import SGDRegressor

In [67]:
model_skGDE = SGDRegressor(tol=0.0001);

In [68]:
model_skGDE.fit(X_train,y_train);

In [71]:
SKGDE_predict = model_skGDE.predict(X_test);

In [77]:
metricas.rmse(SKGDE_predict,y_test)

61130699965467.266

## Com normalização

In [78]:
from ml_libs import Normalizacao

In [79]:
reload(Normalizacao)

<module 'ml_libs.Normalizacao' from '/home/michel/Documentos/Machine-Learning/Lista 5/ml_libs/Normalizacao.py'>

Normalizando os conjutos X de treino e teste

In [97]:
X_train_normal = Normalizacao.normalize(X_train)
X_test_normal = Normalizacao.normalize(X_test)

In [83]:
np.average(X_train_normal),np.average(X_test_normal)

(0.3883534375575344, 0.38610762979178154)

#### Regressão Linear com Gradiente Descendente Estocastico

In [92]:
RLGD2model = Regressao_linear_gd.LinearRegressionGD(eta = 0.1, n_iter = 20);
RLGD2model.fit(X_train_normal,y_train)
RLGD2pred = modelRLGD2.predict(X_test_normal)

In [93]:
metricas.rmse(y_test,RLGD2pred)

6.171711325331826e+43

#### Regressão linear do Scikit-learning

In [98]:
SKRLmodel = LinearRegression()

In [100]:
SKRLmodel.fit(X_train_normal,y_train);

In [101]:
SKRLpred = SKRLmodel.predict(X_test_normal)

In [102]:
metricas.rmse(y_test,SKRLpred)

5.591819387842729

#### Gradiente Descendente Estocástico Regressor do Scikit-Learning

In [106]:
SKGDERmodel = SGDRegressor(tol=0.001)

In [118]:
SKGDERmodel.fit(X_train_normal,y_train)
SKGDERpred = SKGDERmodel.predict(X_test_normal)

In [119]:
metricas.rmse(y_test,SKGDERpred)

5.574565911806708

## Com Standardização

In [124]:
X_train_stand = Normalizacao.standardize(X_train)
X_test_stand = Normalizacao.standardize(X_test)

In [125]:
np.average(X_train_normal),np.average(X_test_stand)

(0.3883534375575344, -1.6361181415528622e-16)

#### Regressão Linear com Gradiente Descendente Estocastico¶

In [131]:
RLGD_s_model = Regressao_linear_gd.LinearRegressionGD(eta = 0.1, n_iter = 20);
RLGD_s_model.fit(X_train_stand,y_train)
RLGD_s_pred = RLGD_s_model.predict(X_test_stand)

In [132]:
metricas.rmse(y_test,RLGD_s_pred)

3.0105485035392498e+47

#### Regressão linear do Scikit-learning

In [133]:
SKRL_s_model = LinearRegression()
SKRL_s_model.fit(X_train_stand,y_train);
SKRL_s_pred = SKRL_s_model.predict(X_test_stand);

In [134]:
metricas.rmse(y_test,SKRL_s_pred)

5.386312922608289

#### Gradiente Descendente Estocástico Regressor do Scikit-Learning

In [135]:
SKGDER_s_model = SGDRegressor(tol=0.001)
SKGDER_s_model.fit(X_train_stand, y_train)
SKGDER_s_pred = SKGDER_s_model.predict(X_test_stand)

In [138]:
metricas.rmse(y_test, SKGDER_s_pred)

5.404745411674031